In [2]:
# Displays figures in line
%matplotlib inline

# Import dependencies 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests 
import json
plt.style.use('ggplot')

from pprint import pprint

In [3]:
url = "https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Public_Safety_WebMercator/MapServer/24/query?where=1%3D1&outFields=*&outSR=4326&f=json"

In [4]:
response = requests.get(url).json()

In [5]:
data = response['features']
crash_data = []

In [6]:
i = 0
for x in data:
    crash_data.append(data[i]['attributes'])
    i = i + 1

In [8]:
master_df = pd.DataFrame(crash_data)
master_df.head()

,ADDRESS,BICYCLISTSIMPAIRED,CCN,CRIMEID,DRIVERSIMPAIRED,EVENTID,FATAL_BICYCLIST,FATAL_DRIVER,FATAL_PEDESTRIAN,FROMDATE,...,TOTAL_GOVERNMENT,TOTAL_PEDESTRIANS,TOTAL_TAXIS,TOTAL_VEHICLES,UNKNOWNINJURIES_BICYCLIST,UNKNOWNINJURIES_DRIVER,UNKNOWNINJURIES_PEDESTRIAN,WARD,XCOORD,YCOORD
0,MINNESOTA AVE NE & EASTERN AVE NE\r\n,0,15184403,26761381,0,{74BF3FC1-4D29-41FC-AB56-2CF656822E0D},0,0,0,1447804800000,...,0,0,0,2,0,0,0,Ward 7,405942.489,138023.869
1,3810 SOUTHERN AVENUE SE,0,17066149,27285672,0,{29653309-EA89-403D-878D-CE2F2B78A2BF},0,0,0,1492819200000,...,0,0,0,2,0,0,0,Ward 7,404346.945,132683.743
2,4600 RESERVOIR ROAD NW,0,17066152,27285673,0,{E91D766E-BB91-4266-B885-F6DD3705ABE1},0,0,0,1492819200000,...,0,0,0,2,0,0,0,Ward 3,392536.865,138328.310
3,12TH ST NE & BUCHANAN ST NE\r\nGO\n,0,15184561,26761387,0,{5E3844B6-B27D-4C5E-B2B1-22E371C4D239},0,0,0,1447804800000,...,0,1,0,1,0,0,0,Ward 5,400846.181,142036.679
4,3900 COLE BOULEVARD SE,0,14183443,25188528,0,{AC54FDC4-90A3-4B0D-BFAA-E552D5191496},0,0,0,1416960000000,...,0,0,0,2,0,0,0,Ward 8,400745.114,129865.986
